# Worked Examples of State Classification: Pure/Mixed × Single/Bipartite × Separable/Entangled

This notebook provides concrete examples and short derivations for single qubit systems (pure and mixed), bipartite systems (pure and mixed), and entanglement.

It uses only the standard python library `numpy`.

Wayne Dam \
Meetup: Physics with Friends - Quantum Information Theory \
Oct. 2025


In [49]:
import numpy as np
# import plotly.graph_objects as go
# from IPython.display import display


In [50]:

# === Pretty printing helpers: drop tiny imaginary parts and suppress small-exponent floats ===
import numpy as np

def _clean(x, tol=1e-12):
    # Drop tiny imaginary parts. For arrays: cast to real if *all* imag parts are tiny;
    # otherwise zero-out tiny imaginary entries elementwise while keeping genuine complex entries.
    if np.isscalar(x):
        return np.real_if_close(x)
    a = np.asarray(x)
    if np.iscomplexobj(a):
        if np.all(np.abs(a.imag) < tol):
            return a.real
        b = a.copy()
        b.imag[np.abs(b.imag) < tol] = 0.0
        return b
    return a

def pp(*args, **kwargs):
    # Clean results then print, so arrays/scalars don't show '+0.j' or tiny 1e-17 noise.
    cleaned = [_clean(arg) for arg in args]
    print(*cleaned, **kwargs)

# Nicer numeric formatting in general:
np.set_printoptions(precision=6, suppress=True)


## Examples and Checks

Examples and checks (purity, PPT/negativity for 2-qubit states, Schmidt rank for pure bipartite states).

In [51]:
import numpy as np

def dag(x):
    return np.conjugate(x.T)

def dm(psi):
    # Density matrix from state vector |psi>
    psi = psi.reshape(-1,1)
    return psi @ dag(psi)

def kron(*args):
    out = np.array([[1.0+0.0j]])
    for a in args:
        out = np.kron(out, a)
    return out

def purity(rho):
    return float(np.real(np.trace(rho @ rho)))

def partial_transpose_2x2(rho, sys='B'):
    # Partial transpose for 2-qubit systems with ordering |00>,|01>,|10>,|11>.
    # sys: 'A' or 'B' -- which subsystem to transpose.
    rho = rho.reshape(2,2,2,2)  # indices iA,iB,jA,jB
    if sys.upper() == 'A':
        rho_pt = np.transpose(rho, (2,1,0,3))
    else:
        rho_pt = np.transpose(rho, (0,3,2,1))
    return rho_pt.reshape(4,4)

def negativity(rho, sys='B'):
    # Sum of absolute values of negative eigenvalues of the partial transpose
    pt = partial_transpose_2x2(rho, sys=sys)
    evals = np.linalg.eigvalsh(pt)
    neg_eigs = evals[evals < 0]
    return float(np.sum(np.abs(neg_eigs)))

def schmidt_rank_two_qubit_state(psi):
    # Schmidt rank from singular values of reshaped 2x2 pure state
    M = psi.reshape(2,2)
    s = np.linalg.svd(M, compute_uv=False)
    return int(np.sum(s > 1e-12)), s

In [52]:
# Standard basis kets: |0>, |1>.
zero = np.array([1,0], dtype=complex)
one  = np.array([0,1], dtype=complex)

# Bell states
phi_plus = (kron(zero, zero) + kron(one, one)) / np.sqrt(2)   # |Φ+>
psi_minus = (kron(zero, one) - kron(one, zero)) / np.sqrt(2)  # |Ψ->

In [53]:
# Examples
examples = {}

# Single–Pure (N/A entanglement)
psi_single_pure = zero  # |0>
rho_single_pure = dm(psi_single_pure)
examples["Single–Pure (N/A): |0>"] = rho_single_pure

# Single–Mixed (N/A entanglement)
rho_single_mixed = 0.5 * np.eye(2)
examples["Single–Mixed (N/A): ρ = I/2"] = rho_single_mixed

# Bipartite–Pure Separable: |0>⊗|1>
psi_prod = kron(zero, one)
rho_prod = dm(psi_prod)
examples["Bipartite–Pure Separable: |0>⊗|1>"] = rho_prod

# Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> )
rho_phi_plus = dm(phi_plus)
examples["Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> )"] = rho_phi_plus

# Bipartite–Mixed Separable: 1/2(|00><00| + |11><11|)
rho_00 = dm(kron(zero, zero))
rho_11 = dm(kron(one, one))
rho_mixed_sep = 0.5*(rho_00 + rho_11)
examples["Bipartite–Mixed Separable: 1/2(|00><00| + |11><11|"] = rho_mixed_sep

# Bipartite–Mixed Entangled: Werner state with p=0.6 (> 1/3)
p = 0.6
rho_werner = p*dm(psi_minus) + (1-p)*(np.eye(4)/4)
examples["Bipartite–Mixed Entangled (Werner, p=0.6): p*dm(|Φ-><Φ-|) + (1-p) I/4)"] = rho_werner


## Worked Examples

Each example below is computed in **its own cell**, repeating the same calculations.


### Example: Single–Pure (N/A): |0>

We will compute:
- Dimension, trace, purity \(\mathrm{Tr}(\rho^2)\), and matrix rank
- For 4×4 states: **negativity** (PPT criterion in 2×2)
- If the 4×4 state is pure (purity ≈ 1): **Schmidt singular values** and **Schmidt rank**


In [54]:
# --- NOTE ---
# Worked example: Single–Pure (N/A): |0>

name = 'Single–Pure (N/A): |0>'
rho = examples[name]

pp("\n=== Single–Pure (N/A): |0> ===")
pp("ρ (density matrix):")
pp(rho)

# --- Basic invariants ---
pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

# --- Two-qubit specific: Negativity (PPT) ---
if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

# --- Two-qubit pure states: Schmidt decomposition ---
if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)  # eigen-decomposition
    psi = evecs[:, np.argmax(evals)]    # eigenvector for eigenvalue 1 (the pure state)
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Single–Pure (N/A): |0> ===
ρ (density matrix):
[[1. 0.]
 [0. 0.]]
Dim: 2
Trace: 1.0
Purity Tr(rho^2): 1.0
Matrix rank: 1



**What to notice for _Single–Pure (N/A): |0>_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Example: Single–Mixed (N/A): ρ = I/2

We will compute:
- Dimension, trace, purity \(\mathrm{Tr}(\rho^2)\), and matrix rank
- For 4×4 states: **negativity** (PPT criterion in 2×2)
- If the 4×4 state is pure (purity ≈ 1): **Schmidt singular values** and **Schmidt rank**


In [55]:
# --- NOTE ---
# Worked example: Single–Mixed (N/A): ρ = I/2

name = 'Single–Mixed (N/A): ρ = I/2'
rho = examples[name]

pp("\n=== Single–Mixed (N/A): ρ = I/2 ===")
pp("ρ (density matrix):")
pp(rho)

# --- Basic invariants ---
pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

# --- Two-qubit specific: Negativity (PPT) ---
if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

# --- Two-qubit pure states: Schmidt decomposition ---
if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)  # eigen-decomposition
    psi = evecs[:, np.argmax(evals)]    # eigenvector for eigenvalue 1 (the pure state)
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Single–Mixed (N/A): ρ = I/2 ===
ρ (density matrix):
[[0.5 0. ]
 [0.  0.5]]
Dim: 2
Trace: 1.0
Purity Tr(rho^2): 0.5
Matrix rank: 2



**What to notice for _Single–Mixed (N/A): ρ = I/2_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Example: Bipartite–Pure Separable: |0>⊗|1>

We will compute:
- Dimension, trace, purity \(\mathrm{Tr}(\rho^2)\), and matrix rank
- For 4×4 states: **negativity** (PPT criterion in 2×2)
- If the 4×4 state is pure (purity ≈ 1): **Schmidt singular values** and **Schmidt rank**


In [56]:
# --- NOTE ---
# Worked example: Bipartite–Pure Separable: |0>⊗|1>

name = 'Bipartite–Pure Separable: |0>⊗|1>'
rho = examples[name]

pp("\n=== Bipartite–Pure Separable: |0>⊗|1> ===")
pp("ρ (density matrix):")
pp(rho)

# --- Basic invariants ---
pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

# --- Two-qubit specific: Negativity (PPT) ---
if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

# --- Two-qubit pure states: Schmidt decomposition ---
if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)  # eigen-decomposition
    psi = evecs[:, np.argmax(evals)]    # eigenvector for eigenvalue 1 (the pure state)
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Bipartite–Pure Separable: |0>⊗|1> ===
ρ (density matrix):
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
Dim: 4
Trace: 1.0
Purity Tr(rho^2): 1.0
Matrix rank: 1
Negativity (PPT detects entanglement if >0 in 2x2): 0.0
PPT says: Separable (2x2 case)
Schmidt singular values: [1. 0.]
Schmidt rank: 1



**What to notice for _Bipartite–Pure Separable: |0>⊗|1>_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Example: Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> )

We will compute:
- Dimension, trace, purity \(\mathrm{Tr}(\rho^2)\), and matrix rank
- For 4×4 states: **negativity** (PPT criterion in 2×2)
- If the 4×4 state is pure (purity ≈ 1): **Schmidt singular values** and **Schmidt rank**


In [57]:
# --- NOTE ---
# Worked example: Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> )

name = 'Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> )'
rho = examples[name]

pp("\n=== Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> ) ===")
pp("ρ (density matrix):")
pp(rho)

# --- Basic invariants ---
pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

# --- Two-qubit specific: Negativity (PPT) ---
if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

# --- Two-qubit pure states: Schmidt decomposition ---
if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)  # eigen-decomposition
    psi = evecs[:, np.argmax(evals)]    # eigenvector for eigenvalue 1 (the pure state)
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> ) ===
ρ (density matrix):
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
Dim: 4
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.9999999999999996
Matrix rank: 1
Negativity (PPT detects entanglement if >0 in 2x2): 0.4999999999999999
PPT says: Entangled
Schmidt singular values: [0.70710678 0.70710678]
Schmidt rank: 2



**What to notice for _Bipartite–Pure Entangled: |Φ+> = 1/√2 ( |01> + |10> )_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Example: Bipartite–Mixed Separable: 1/2(|00><00| + |11><11|

We will compute:
- Dimension, trace, purity \(\mathrm{Tr}(\rho^2)\), and matrix rank
- For 4×4 states: **negativity** (PPT criterion in 2×2)
- If the 4×4 state is pure (purity ≈ 1): **Schmidt singular values** and **Schmidt rank**


In [58]:
# --- NOTE ---
# Worked example: Bipartite–Mixed Separable: 1/2(|00><00| + |11><11|

name = 'Bipartite–Mixed Separable: 1/2(|00><00| + |11><11|'
rho = examples[name]

pp("\n=== Bipartite–Mixed Separable: 1/2(|00><00| + |11><11| ===")
pp("ρ (density matrix):")
pp(rho)

# --- Basic invariants ---
pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

# --- Two-qubit specific: Negativity (PPT) ---
if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

# --- Two-qubit pure states: Schmidt decomposition ---
if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)  # eigen-decomposition
    psi = evecs[:, np.argmax(evals)]    # eigenvector for eigenvalue 1 (the pure state)
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Bipartite–Mixed Separable: 1/2(|00><00| + |11><11| ===
ρ (density matrix):
[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0.5]]
Dim: 4
Trace: 1.0
Purity Tr(rho^2): 0.5
Matrix rank: 2
Negativity (PPT detects entanglement if >0 in 2x2): 0.0
PPT says: Separable (2x2 case)



**What to notice for _Bipartite–Mixed Separable: 1/2(|00><00| + |11><11|_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Example: Bipartite–Mixed Entangled (Werner, p=0.6): p*dm(|Φ-><Φ-|) + (1-p) I/4)

We will compute:
- Dimension, trace, purity \(\mathrm{Tr}(\rho^2)\), and matrix rank
- For 4×4 states: **negativity** (PPT criterion in 2×2)
- If the 4×4 state is pure (purity ≈ 1): **Schmidt singular values** and **Schmidt rank**


In [59]:
# --- NOTE ---
# Worked example: Bipartite–Mixed Entangled (Werner, p=0.6): p*dm(|Φ-><Φ-|) + (1-p) I/4)

name = 'Bipartite–Mixed Entangled (Werner, p=0.6): p*dm(|Φ-><Φ-|) + (1-p) I/4)'
rho = examples[name]

pp("\n=== Bipartite–Mixed Entangled (Werner, p=0.6): p*dm(|Φ-><Φ-|) + (1-p) I/4) ===")
pp("ρ (density matrix):")
pp(rho)

# --- Basic invariants ---
pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

# --- Two-qubit specific: Negativity (PPT) ---
if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

# --- Two-qubit pure states: Schmidt decomposition ---
if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)  # eigen-decomposition
    psi = evecs[:, np.argmax(evals)]    # eigenvector for eigenvalue 1 (the pure state)
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Bipartite–Mixed Entangled (Werner, p=0.6): p*dm(|Φ-><Φ-|) + (1-p) I/4) ===
ρ (density matrix):
[[ 0.1  0.   0.   0. ]
 [ 0.   0.4 -0.3  0. ]
 [ 0.  -0.3  0.4  0. ]
 [ 0.   0.   0.   0.1]]
Dim: 4
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.5199999999999998
Matrix rank: 4
Negativity (PPT detects entanglement if >0 in 2x2): 0.19999999999999993
PPT says: Entangled



**What to notice for _Bipartite–Mixed Entangled (Werner, p=0.6): p*dm(|Φ-><Φ-|) + (1-p) I/4)_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


## Derivations (inline LaTeX)

### Pure vs. Mixed via $\operatorname{Tr}(\rho^2)$
Let $\rho$ be a density operator on a finite-dimensional Hilbert space.

**Claim.** $\rho$ is pure $\iff \operatorname{Tr}(\rho^2)=1$.

**Proof.** If $\rho=|\psi\rangle\!\langle\psi|$, then $\rho^2=\rho$ and $\operatorname{Tr}(\rho^2)=\operatorname{Tr}(\rho)=1$.
Conversely, with $\rho=\sum_i \lambda_i |i\rangle\!\langle i|$ ($\lambda_i\ge0$, $\sum_i\lambda_i=1$), we have $\operatorname{Tr}(\rho^2)=\sum_i\lambda_i^2\le 1$, with equality iff one eigenvalue is 1 (rank 1). $\square$

### Schmidt Decomposition and Pure-State Entanglement
For $|\psi\rangle\in \mathcal H_A\otimes\mathcal H_B$, reshape coefficients into a matrix $M$ and take the SVD $M=U\,\mathrm{diag}(\sqrt{\lambda_i})\,V^\dagger$.
This yields $|\psi\rangle=\sum_i\sqrt{\lambda_i}\,|u_i\rangle\otimes|v_i\rangle$; the number of nonzero $\lambda_i$ is the **Schmidt rank**. Separable $\iff$ rank $=1$; entangled $\iff$ rank $>1$.

### Peres–Horodecki (PPT) Criterion in $2\times2$ and $2\times3$
If $\rho_{AB}$ is separable then $\rho_{AB}^{T_B}\ge0$. In $2\times2$ and $2\times3$, PPT is also sufficient; in higher dimensions, PPT is only necessary (PPT-entangled states exist).

### Two-Qubit Werner State Threshold
For $\rho_W(p)=p|\Psi^-\rangle\!\langle\Psi^-|+(1-p)\tfrac{I}{4}$, the eigenvalues of $\rho_W(p)^{T_B}$ are $\{\tfrac{1-3p}{4},\tfrac{1+p}{4},\tfrac{1+p}{4},\tfrac{1+p}{4}\}$, so negativity appears (and the state is entangled) iff $p>\tfrac{1}{3}$.

## Citations
- **Nielsen & Chuang (N&C)**, *Quantum Computation and Quantum Information*, 2nd ed. — Ch. 2 (states), Sec. 2.5 (Schmidt).
- **Watrous**, *The Theory of Quantum Information* — Early chapters (states, Schmidt), entanglement & PPT.
- **Wilde**, *Quantum Information Theory*, 2nd ed. — Entanglement theory, PPT, Werner states.
- **Khatri–Lami–Wilde**, *Principles of Quantum Communication Theory* — Modern treatment of states, channels, entanglement.
- **Manenti & Motta**, *Quantum Information Science* — Density matrices, measurements, examples.
- **Jacobs**, *Quantum Measurement Theory and its Applications* — System–probe measurement framework.

Original papers: A. Peres, *Phys. Rev. Lett.* **77**, 1413 (1996); M. Horodecki, P. Horodecki, R. Horodecki, *Phys. Lett. A* **223**, 1–8 (1996).


## Additional Practice Examples (two per case)

Below we add two **new** examples for each major case:
- Two-qubit **pure entangled**
- Two-qubit **pure separable**
- Two-qubit **mixed separable**
- Two-qubit **mixed entangled**
- Single-qubit **pure**
- Single-qubit **mixed**

Each example is fully commented and includes a **What to notice** block with a quick checklist.


In [60]:
# --- SETUP for Additional Practice Examples ---
# This cell defines convenient basis vectors, projectors, and helpers.
# It also ensures `examples` exist, appending all new practice states into it.

import numpy as np

# Computational basis for 1 qubit
zero = np.array([1, 0], dtype=complex)
one  = np.array([0, 1], dtype=complex)

# PLUS / MINUS and eigenstates of sigma_y
plus  = (zero + one) / np.sqrt(2)
minus = (zero - one) / np.sqrt(2)
yplus  = (zero + 1j*one) / np.sqrt(2)
yminus = (zero - 1j*one) / np.sqrt(2)

proj = lambda v: np.outer(v, np.conjugate(v))
kron = np.kron
I2 = np.eye(2, dtype=complex)
I4 = np.eye(4, dtype=complex)

# Ensure examples dict exists
try:
    examples
except NameError:
    examples = {}

# A small helper to add and announce a state
def add_example(name, rho):
    # Force Hermiticity + trace normalization (in case of rounding)
    rho = 0.5*(rho + rho.conj().T)
    tr = np.trace(rho)
    if not np.isclose(tr, 1.0):
        rho = rho / tr
    examples[name] = rho


### Two-qubit — Pure Entangled

#### Example: Bell Ψ+ (|01>+|10>)/√2

A standard Bell state with maximal entanglement; expect purity=1, negativity>0, Schmidt rank 2.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [61]:
# --- NOTE ---
# Two-qubit — Pure Entangled — Bell Ψ+ (|01>+|10>)/√2
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
psi = (kron(zero, one) + kron(one, zero)) / np.sqrt(2)
rho = proj(psi)

add_example("Bell Ψ+ (|01>+|10>)/√2", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "Bell Ψ+ (|01>+|10>)/√2"
rho = examples[name]

pp("\n=== Bell Ψ+ (|01>+|10>)/√2 ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Bell Ψ+ (|01>+|10>)/√2 ===
ρ (density matrix):
[[0.  0.  0.  0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.5 0.5 0. ]
 [0.  0.  0.  0. ]]
Dim: 4
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.9999999999999996
Matrix rank: 1
Negativity (PPT detects entanglement if >0 in 2x2): 0.4999999999999999
PPT says: Entangled
Schmidt singular values: [0.70710678 0.70710678]
Schmidt rank: 2



**What to notice for _Bell Ψ+ (|01>+|10>)/√2_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


#### Example: Partially entangled cos(π/6)|00> + sin(π/6)|11>

A tunable pure entangled state; purity=1, Schmidt rank 2 (since both amplitudes nonzero), nonzero negativity.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [62]:
# --- NOTE ---
# Two-qubit — Pure Entangled — Partially entangled cos(π/6)|00> + sin(π/6)|11>
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
theta = np.pi/6
psi = np.cos(theta)*kron(zero, zero) + np.sin(theta)*kron(one, one)
rho = proj(psi)

add_example("Partially entangled cos(π/6)|00> + sin(π/6)|11>", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "Partially entangled cos(π/6)|00> + sin(π/6)|11>"
rho = examples[name]

pp("\n=== Partially entangled cos(π/6)|00> + sin(π/6)|11> ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Partially entangled cos(π/6)|00> + sin(π/6)|11> ===
ρ (density matrix):
[[0.75      0.        0.        0.4330127]
 [0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.4330127 0.        0.        0.25     ]]
Dim: 4
Trace: 1.0
Purity Tr(rho^2): 1.0000000000000002
Matrix rank: 1
Negativity (PPT detects entanglement if >0 in 2x2): 0.4330127018922193
PPT says: Entangled
Schmidt singular values: [0.8660254 0.5      ]
Schmidt rank: 2



**What to notice for _Partially entangled cos(π/6)|00> + sin(π/6)|11>_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Two-qubit — Pure Separable

#### Example: |0> ⊗ |+>

Product of single-qubit pure states; expect purity=1, negativity=0, Schmidt rank 1.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [63]:
# --- NOTE ---
# Two-qubit — Pure Separable — |0> ⊗ |+>
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
psi = kron(zero, plus)
rho = proj(psi)

add_example("|0> ⊗ |+>", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "|0> ⊗ |+>"
rho = examples[name]

pp("\n=== |0> ⊗ |+> ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== |0> ⊗ |+> ===
ρ (density matrix):
[[0.5 0.5 0.  0. ]
 [0.5 0.5 0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]]
Dim: 4
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.9999999999999996
Matrix rank: 1
Negativity (PPT detects entanglement if >0 in 2x2): 0.0
PPT says: Separable (2x2 case)
Schmidt singular values: [1. 0.]
Schmidt rank: 1



**What to notice for _|0> ⊗ |+>_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


#### Example: |1> ⊗ (cos(π/3)|0> + sin(π/3)|1>)

Another product pure state; expect purity=1, negativity=0, Schmidt rank 1.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [64]:
# --- NOTE ---
# Two-qubit — Pure Separable — |1> ⊗ (cos(π/3)|0> + sin(π/3)|1>)
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
phi = np.pi/3
b = np.cos(phi)*zero + np.sin(phi)*one
psi = kron(one, b)
rho = proj(psi)

add_example("|1> ⊗ (cos(π/3)|0> + sin(π/3)|1>)", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "|1> ⊗ (cos(π/3)|0> + sin(π/3)|1>)"
rho = examples[name]

pp("\n=== |1> ⊗ (cos(π/3)|0> + sin(π/3)|1>) ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== |1> ⊗ (cos(π/3)|0> + sin(π/3)|1>) ===
ρ (density matrix):
[[0.        0.        0.        0.       ]
 [0.        0.        0.        0.       ]
 [0.        0.        0.25      0.4330127]
 [0.        0.        0.4330127 0.75     ]]
Dim: 4
Trace: 1.0
Purity Tr(rho^2): 1.0
Matrix rank: 1
Negativity (PPT detects entanglement if >0 in 2x2): 2.7755575615628914e-17
PPT says: Separable (2x2 case)
Schmidt singular values: [1. 0.]
Schmidt rank: 1



**What to notice for _|1> ⊗ (cos(π/3)|0> + sin(π/3)|1>)_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Two-qubit — Mixed Separable

#### Example: 0.7|00><00| + 0.3|11><11|

Classical mixture of two product states; PPT gives 0 negativity (separable), purity<1, rank>1.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [65]:
# --- NOTE ---
# Two-qubit — Mixed Separable — 0.7|00><00| + 0.3|11><11|
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
rho = 0.7*proj(kron(zero, zero)) + 0.3*proj(kron(one, one))

add_example("0.7|00><00| + 0.3|11><11|", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "0.7|00><00| + 0.3|11><11|"
rho = examples[name]

pp("\n=== 0.7|00><00| + 0.3|11><11| ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== 0.7|00><00| + 0.3|11><11| ===
ρ (density matrix):
[[0.7 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0.3]]
Dim: 4
Trace: 1.0
Purity Tr(rho^2): 0.58
Matrix rank: 2
Negativity (PPT detects entanglement if >0 in 2x2): 0.0
PPT says: Separable (2x2 case)



**What to notice for _0.7|00><00| + 0.3|11><11|_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


#### Example: 0.5|01><01| + 0.5|10><10|

Another classical mixture; separable with negativity 0, purity<1, rank>1.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [66]:
# --- NOTE ---
# Two-qubit — Mixed Separable — 0.5|01><01| + 0.5|10><10|
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
rho = 0.5*proj(kron(zero, one)) + 0.5*proj(kron(one, zero))

add_example("0.5|01><01| + 0.5|10><10|", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "0.5|01><01| + 0.5|10><10|"
rho = examples[name]

pp("\n=== 0.5|01><01| + 0.5|10><10| ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== 0.5|01><01| + 0.5|10><10| ===
ρ (density matrix):
[[0.  0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  0. ]]
Dim: 4
Trace: 1.0
Purity Tr(rho^2): 0.5
Matrix rank: 2
Negativity (PPT detects entanglement if >0 in 2x2): 0.0
PPT says: Separable (2x2 case)



**What to notice for _0.5|01><01| + 0.5|10><10|_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Two-qubit — Mixed Entangled

#### Example: Werner(0.8)  0.8|Φ+><Φ+| + 0.2*I/4

Werner state is entangled for p>1/3; expect negativity>0 (2×2 PPT detects entanglement).

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [67]:
# --- NOTE ---
# Two-qubit — Mixed Entangled — Werner(0.8)  0.8|Φ+><Φ+| + 0.2*I/4
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
phi_plus = (kron(zero, zero) + kron(one, one)) / np.sqrt(2)
rho = 0.8*proj(phi_plus) + 0.2*(np.eye(4, dtype=complex)/4)

add_example("Werner(0.8)  0.8|Φ+><Φ+| + 0.2*I/4", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "Werner(0.8)  0.8|Φ+><Φ+| + 0.2*I/4"
rho = examples[name]

pp("\n=== Werner(0.8)  0.8|Φ+><Φ+| + 0.2*I/4 ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Werner(0.8)  0.8|Φ+><Φ+| + 0.2*I/4 ===
ρ (density matrix):
[[0.45 0.   0.   0.4 ]
 [0.   0.05 0.   0.  ]
 [0.   0.   0.05 0.  ]
 [0.4  0.   0.   0.45]]
Dim: 4
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.7299999999999996
Matrix rank: 4
Negativity (PPT detects entanglement if >0 in 2x2): 0.3499999999999999
PPT says: Entangled



**What to notice for _Werner(0.8)  0.8|Φ+><Φ+| + 0.2*I/4_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


#### Example: Isotropic(0.6)  0.6|Φ+><Φ+| + 0.4*I/4

Isotropic state also entangled for α>1/3 in 2×2; expect negativity>0.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [68]:
# --- NOTE ---
# Two-qubit — Mixed Entangled — Isotropic(0.6)  0.6|Φ+><Φ+| + 0.4*I/4
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
phi_plus = (kron(zero, zero) + kron(one, one)) / np.sqrt(2)
rho = 0.6*proj(phi_plus) + 0.4*(np.eye(4, dtype=complex)/4)

add_example("Isotropic(0.6)  0.6|Φ+><Φ+| + 0.4*I/4", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "Isotropic(0.6)  0.6|Φ+><Φ+| + 0.4*I/4"
rho = examples[name]

pp("\n=== Isotropic(0.6)  0.6|Φ+><Φ+| + 0.4*I/4 ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== Isotropic(0.6)  0.6|Φ+><Φ+| + 0.4*I/4 ===
ρ (density matrix):
[[0.4 0.  0.  0.3]
 [0.  0.1 0.  0. ]
 [0.  0.  0.1 0. ]
 [0.3 0.  0.  0.4]]
Dim: 4
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.5199999999999998
Matrix rank: 4
Negativity (PPT detects entanglement if >0 in 2x2): 0.19999999999999996
PPT says: Entangled



**What to notice for _Isotropic(0.6)  0.6|Φ+><Φ+| + 0.4*I/4_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Single-qubit — Pure

#### Example: |+><+| (X-eigenstate)

Pure single-qubit state; purity=1, rank 1.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [69]:
# --- NOTE ---
# Single-qubit — Pure — |+><+| (X-eigenstate)
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
rho = proj(plus)

add_example("|+><+| (X-eigenstate)", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "|+><+| (X-eigenstate)"
rho = examples[name]

pp("\n=== |+><+| (X-eigenstate) ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== |+><+| (X-eigenstate) ===
ρ (density matrix):
[[0.5 0.5]
 [0.5 0.5]]
Dim: 2
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.9999999999999996
Matrix rank: 1



**What to notice for _|+><+| (X-eigenstate)_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


#### Example: |y+><y+| (Y-eigenstate)

Another pure single-qubit state; purity=1, rank 1.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [70]:
# --- NOTE ---
# Single-qubit — Pure — |y+><y+| (Y-eigenstate)
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
rho = proj(yplus)

add_example("|y+><y+| (Y-eigenstate)", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "|y+><y+| (Y-eigenstate)"
rho = examples[name]

pp("\n=== |y+><y+| (Y-eigenstate) ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== |y+><y+| (Y-eigenstate) ===
ρ (density matrix):
[[0.5+0.j  0. -0.5j]
 [0. +0.5j 0.5+0.j ]]
Dim: 2
Trace: 0.9999999999999998
Purity Tr(rho^2): 0.9999999999999996
Matrix rank: 1



**What to notice for _|y+><y+| (Y-eigenstate)_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


### Single-qubit — Mixed

#### Example: 0.8|0><0| + 0.2|1><1|

Classical mixture on Z basis; purity<1, rank up to 2.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [71]:
# --- NOTE ---
# Single-qubit — Mixed — 0.8|0><0| + 0.2|1><1|
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
rho = 0.8*proj(zero) + 0.2*proj(one)

add_example("0.8|0><0| + 0.2|1><1|", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "0.8|0><0| + 0.2|1><1|"
rho = examples[name]

pp("\n=== 0.8|0><0| + 0.2|1><1| ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== 0.8|0><0| + 0.2|1><1| ===
ρ (density matrix):
[[0.8 0. ]
 [0.  0.2]]
Dim: 2
Trace: 1.0
Purity Tr(rho^2): 0.6800000000000002
Matrix rank: 2



**What to notice for _0.8|0><0| + 0.2|1><1|_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)


#### Example: I/2 (maximally mixed)

Completely mixed state; purity=1/2, rank 2.

We'll compute the same diagnostics as before (dimension, trace, purity, rank; plus PPT-negativity and Schmidt data when applicable).


In [72]:
# --- NOTE ---
# Single-qubit — Mixed — I/2 (maximally mixed)
# Construct the state explicitly and add it into `examples` for consistency with earlier cells.

# Define the density matrix ρ for this example:
rho = np.eye(2, dtype=complex)/2

add_example("I/2 (maximally mixed)", rho)

# Now run the standard diagnostics (identical to earlier examples):
name = "I/2 (maximally mixed)"
rho = examples[name]

pp("\n=== I/2 (maximally mixed) ===")
pp("ρ (density matrix):")
pp(rho)

pp("Dim:", rho.shape[0])
pp("Trace:", np.trace(rho))
pp("Purity Tr(rho^2):", purity(rho))

rank = np.linalg.matrix_rank(rho, tol=1e-12)
pp("Matrix rank:", rank)

if rho.shape == (4, 4):
    neg = negativity(rho, sys='B')
    pp("Negativity (PPT detects entanglement if >0 in 2x2):", neg)
    if np.isclose(np.real_if_close(neg), 0.0, atol=1e-12):
        pp("PPT says: Separable (2x2 case)")
    else:
        pp("PPT says: Entangled")

if rho.shape == (4, 4) and np.isclose(purity(rho), 1.0, atol=1e-12):
    evals, evecs = np.linalg.eigh(rho)
    psi = evecs[:, np.argmax(evals)]
    r, svals = schmidt_rank_two_qubit_state(psi)
    pp("Schmidt singular values:", svals)
    pp("Schmidt rank:", r)



=== I/2 (maximally mixed) ===
ρ (density matrix):
[[0.5 0. ]
 [0.  0.5]]
Dim: 2
Trace: 1.0
Purity Tr(rho^2): 0.5
Matrix rank: 2



**What to notice for _I/2 (maximally mixed)_**

- How **dimension** and **matrix rank** relate to purity (rank 1 ⇒ purity = 1 for density matrices).
- Whether the **trace** is exactly 1 (valid states must satisfy this).
- For 4×4 (two-qubit) states:
  - If **negativity** is **> 0** ⇒ entangled (PPT test is decisive for 2×2).
  - If **negativity** is **0** ⇒ PPT-separable in 2×2; often a classical mixture.
- For pure 4×4 states (purity ≈ 1): compare the **Schmidt rank** with entanglement.
  - Schmidt rank 1 ⇒ product (separable) state.
  - Schmidt rank 2 ⇒ entangled state.
  
**Quick checklist (tick after running the code above):**
- [ ] Trace(ρ) = 1  
- [ ] Purity Tr(ρ²) = 1 (pure) or < 1 (mixed)  
- [ ] Rank(ρ): 1 for pure; >1 for mixed  
- [ ] If 4×4: Negativity > 0? (entangled)  
- [ ] If 4×4 & pure: Schmidt rank = 1 (separable) or 2 (entangled)
